# Projeto #1 - Mineração de padrões frequentes

<p style='text-align: justify;'><font size=3.>Análise de acidentes nas rodovias federais brasileira entre 2007 e 2021.</font></p>

> <p style='text-align: justify;'><font size=2.85>Esse projeto utiliza a versão 0.17.0 do mlxtend.</font></p>

In [1]:
import mlxtend
print(mlxtend.__version__)

0.17.0


### Bibliotecas básicas e outros imports

In [2]:
import warnings
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)
np.set_printoptions(suppress=True, precision=3)

%run modules/text.py

### Carregamento dos dados

In [3]:
df = pd.read_csv('dataset/cleaned_acidentes_rodovias_brasileiras_2007_a_2021.csv', low_memory=False, encoding='utf-8')
print(f'Linhas: {df.shape[0]} | Coluna:  {df.shape[1]}')
df.head(1)

Linhas: 1894736 | Coluna:  24


,id,dia_semana,uf,br,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,tipo_veiculo,tipo_envolvido,estado_fisico,data,mes,ano,estacao_do_ano,regiao_pais,estacao,clima
0,83123214.0,quarta,SC,280,Rio negrinho,outras,saida de pista,ignorada,plena noite,decrescente,chuva,simples,curva,urbano,automovel,condutor,ignorada,2014-01-01,1,2014,verão,sul,3,1


### Remoção valores inválidos

In [4]:
df.drop(df[df['tipo_veiculo'] == 'nao identificado'].index, inplace=True)
df.drop(df[df['tipo_veiculo'] == 'nao informado'].index, inplace=True)
df.drop(df[df['classificacao_acidente'] == 'ignorada'].index, inplace=True)
df.drop(df[df['condicao_metereologica'] == 'ignorada'].index, inplace=True)
print(f'Linhas: {df.shape[0]} | Coluna:  {df.shape[1]}')

Linhas: 1844479 | Coluna:  24


### Geração de features

#### Agrupamento tipos de acidentes

In [5]:
df['tipo_acidente'] = df['tipo_acidente'].apply(mapTiposDeAcidentes)

#### Agrupamento fase do dia

In [6]:
df['fase_dia'] = df['fase_dia'].map(mapFase)

#### Agrupamento dias da semana

In [7]:
df['dia_semana'] = df['dia_semana'].apply(mapDiasDaSemana)

#### Agrupamento condição meteorológica

In [8]:
df['condicao_clima'] = df['condicao_metereologica'].map(mapClima)

### Agrupamento tamanho veículos

In [9]:
df['porte_veiculo'] = df['tipo_veiculo'].apply(mapTamanhoVeiculos)

### Seleção dos recursos para análise

In [10]:
cols = [
        'classificacao_acidente',
        'tipo_acidente', 
        'fase_dia',
        'condicao_clima', 
        'tracado_via',
        'porte_veiculo'
        ]
data = df[cols]
data.head()

,classificacao_acidente,tipo_acidente,fase_dia,condicao_clima,tracado_via,porte_veiculo
1,sem vitimas,colisão,noite,tempo ruim,reta,pequeno porte
2,sem vitimas,atropelamento,noite,tempo bom,reta,pequeno porte
3,sem vitimas,colisão,noite,tempo bom,cruzamento,pequeno porte
4,com vitimas feridas,colisão,noite,nublado,reta,pequeno porte
5,sem vitimas,colisão,noite,nublado,reta,pequeno porte


### Agrupamento

#### 1º cluster: Sem vítimas

In [11]:
sem_vitimas = data.query('classificacao_acidente=="sem vitimas"')
sem_vitimas.drop(['classificacao_acidente'], axis=1, inplace=True)
print(f'Linhas: {sem_vitimas.shape[0]} | Coluna:  {sem_vitimas.shape[1]}')

Linhas: 968151 | Coluna:  5


#### 2º cluster: Vítimas feridas



In [12]:
vitimas_feridas = data.query('classificacao_acidente=="com vitimas feridas"')
vitimas_feridas.drop(['classificacao_acidente'], axis=1, inplace=True)
print(f'Linhas: {vitimas_feridas.shape[0]} | Coluna:  {vitimas_feridas.shape[1]}')

Linhas: 797703 | Coluna:  5


#### 3º cluster: Vítimas fatais

In [13]:
vitimas_fatais = data.query('classificacao_acidente=="com vitimas fatais"')
vitimas_fatais.drop(['classificacao_acidente'], axis=1, inplace=True)
print(f'Linhas: {vitimas_fatais.shape[0]} | Coluna:  {vitimas_fatais.shape[1]}')

Linhas: 78625 | Coluna:  5


## Mineração das regras

#### Enconder da base

In [14]:
def transactionEncoderData(dados:np.ndarray) -> pd.DataFrame:
    transEncoder = TransactionEncoder()
    a_data = transEncoder.fit(dados).transform(dados)
    dados = pd.DataFrame(a_data, columns=transEncoder.columns_)
    return dados*1

#### Itens frequentes
> <p style='text-align: justify;'><font size=2.85>Fpgrowth foi escolhido por ser mais eficiente</font></p>

In [15]:
def getFrequentItemsets(encoderData:pd.DataFrame, minsup:float=0.3, useColnames:bool=True) -> pd.DataFrame:
    return fpgrowth(encoderData, min_support=minsup, use_colnames=useColnames)

#### Regras

In [16]:
def getAssociationRules(itemsets:pd.DataFrame, metric:str='lift', minThreshold:int=1) -> pd.DataFrame:
    return association_rules(itemsets, metric=metric, min_threshold=minThreshold)

### Acidentes sem vítimas

In [17]:
sem_vitimas = transactionEncoderData(sem_vitimas.values)
frequent_itemsets = getFrequentItemsets(sem_vitimas)
rules_sem_vitimas = getAssociationRules(frequent_itemsets)
rules_sem_vitimas = rules_sem_vitimas.sort_values(by='lift', ascending=False)
rules_sem_vitimas[['antecedents', 'consequents', 'confidence', 'lift', 'leverage', 'conviction']].query('confidence>=0.70').reset_index(drop=True)

,antecedents,consequents,confidence,lift,leverage,conviction
0,"(dia, reta)",(colisão),0.798857,1.122588,0.040903,1.433702
1,"(dia, tempo bom)",(colisão),0.797706,1.120971,0.036762,1.425544
2,"(pequeno porte, tempo bom)",(colisão),0.791467,1.112204,0.031334,1.382896
3,"(colisão, tempo bom)",(dia),0.737754,1.108844,0.033439,1.276146
4,"(colisão, tempo bom)",(reta),0.785558,1.106479,0.034906,1.352525
5,"(reta, tempo bom)",(colisão),0.785705,1.104108,0.034202,1.345716
6,"(reta, pequeno porte)",(colisão),0.781354,1.097993,0.032525,1.318935
7,"(pequeno porte, tempo bom)",(reta),0.769513,1.083880,0.023370,1.258371
8,(tempo bom),(reta),0.760412,1.071061,0.030629,1.210572
9,"(dia, tempo bom)",(reta),0.759582,1.069891,0.021190,1.206391


> <p style='text-align: justify;'><font size=2.85>Nos resultados da análise das regras para acidentes sem vítimas, tivemos duas regras com o maior valor de Lift de 1.12 e elas mostram que há, principalmente, colisões que ocorrem durante o dia e com o tempo bom em pistas com traçado reto. Avaliando as demais regras, temos que a maioria dos acidentes envolvem veículos de pequeno porte sendo o tipo de acidente mais comum as colisões.</font></p>

### Acidentes com vítimas feridas

In [18]:
vitimas_feridas = transactionEncoderData(vitimas_feridas.values)
frequent_itemsets = getFrequentItemsets(vitimas_feridas)
rules_vitimas_feridas = getAssociationRules(frequent_itemsets)
rules_vitimas_feridas = rules_vitimas_feridas.sort_values(by='lift', ascending=False)
rules_vitimas_feridas[['antecedents', 'consequents', 'confidence', 'lift', 'leverage', 'conviction']].query('confidence>=0.60').reset_index(drop=True)

,antecedents,consequents,confidence,lift,leverage,conviction
0,(colisão),(reta),0.719559,1.058877,0.023274,1.142668
1,(reta),(colisão),0.615961,1.058877,0.023274,1.089183
2,(reta),(tempo bom),0.687916,1.050160,0.022328,1.105284
3,(tempo bom),(reta),0.713635,1.050160,0.022328,1.119029
4,(dia),(tempo bom),0.686486,1.047977,0.019321,1.100243
5,(tempo bom),(dia),0.644279,1.047977,0.019321,1.082917
6,(colisão),(tempo bom),0.682275,1.041548,0.015832,1.085660
7,(tempo bom),(colisão),0.605880,1.041548,0.015832,1.061324
8,(colisão),(dia),0.626630,1.019269,0.006891,1.031727
9,(pequeno porte),(dia),0.615903,1.001820,0.000560,1.002913


> <p style='text-align: justify;'><font size=2.85>Para o cluster de vítimas feridas, temos a principal regra com um lift de 1.06 indicando que quando ocorrem colisões em pistas retas é provável que os envolvidos sofram algum tipo de ferimento. Também temos nesse grupo que os acidentes que causam ferimentos ocorrem em condições de tempo bom, durante o período do dia e também envolvem veículos de pequeno porte.</font></p>

### Acidentes com vítimas fatais

In [19]:
vitimas_fatais = transactionEncoderData(vitimas_fatais.values)
frequent_itemsets = getFrequentItemsets(vitimas_fatais)
rules_vitimas_fatais = getAssociationRules(frequent_itemsets)
rules_vitimas_fatais = rules_vitimas_fatais.sort_values(by='lift', ascending=False)
rules_vitimas_fatais[['antecedents', 'consequents', 'confidence', 'lift', 'leverage', 'conviction']].query('confidence>=0.60').reset_index(drop=True)

,antecedents,consequents,confidence,lift,leverage,conviction
0,(noite),(reta),0.735494,1.054720,0.019965,1.144262
1,(dia),(colisão),0.641352,1.052148,0.015156,1.088631
2,(dia),(tempo bom),0.696116,1.044407,0.014112,1.097398
3,(reta),(tempo bom),0.692548,1.039053,0.018151,1.084661
4,(tempo bom),(reta),0.724568,1.039053,0.018151,1.098873
5,(pequeno porte),(reta),0.710476,1.018843,0.005890,1.045385
6,(colisão),(reta),0.706637,1.013339,0.005670,1.031707
7,(reta),(colisão),0.617695,1.013339,0.005670,1.021268


> <p style='text-align: justify;'><font size=2.85>Em acidentes com vítimas fatais, a nossa principal regra, com lift de 1.05, nos mostra que nesse cluster os acidentes com mortes ocorrem, principalmente, durante a noite e em pistas com traçado reto. Boa parte também ocorre no período do dia e com um tempo bom. O tipo de acidente mais recorrente são as colisões e essas ocorrências também envolvem veículos de pequeno porte.</font></p>